<a href="https://colab.research.google.com/github/anicelysantos/book-python-para-analise-de-dados/blob/main/agregacao_operacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [92]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Funcionamento do GroupBy

In [2]:
df = pd.DataFrame({'key1':['a','a','b','b','a'], 'key':['one','two','one','two','one'], 'data1': np.random.randn(5), 'data2': np.random.randn(5)})
df

,key1,key,data1,data2
0,a,one,-1.935819,-0.362509
1,a,two,0.286133,-0.252848
2,b,one,0.103624,0.410228
3,b,two,0.964137,0.426694
4,a,one,2.679130,0.194852


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.343148
b    0.533880
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key']]).mean()
means

key1  key
a     one    0.371655
      two    0.286133
b     one    0.103624
      two    0.964137
Name: data1, dtype: float64

In [6]:
means.unstack()

key,one,two
key1,,
a,0.371655,0.286133
b,0.103624,0.964137


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.286133
            2006    0.103624
Ohio        2005   -0.485841
            2006    2.679130
Name: data1, dtype: float64

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.343148,-0.140169
b,0.533880,0.418461


In [9]:
df.groupby(['key1', 'key']).mean()

data1     data2
key1 key                    
a    one  0.371655 -0.083829
     two  0.286133 -0.252848
b    one  0.103624  0.410228
     two  0.964137  0.426694

In [10]:
df.groupby(['key1', 'key']).size()

key1  key
a     one    2
      two    1
b     one    1
      two    1
dtype: int64

**Iterando por grupos**

In [11]:
for nome, grupo in df.groupby('key1'):
  print(nome)
  print(grupo)

a
  key1  key     data1     data2
0    a  one -1.935819 -0.362509
1    a  two  0.286133 -0.252848
4    a  one  2.679130  0.194852
b
  key1  key     data1     data2
2    b  one  0.103624  0.410228
3    b  two  0.964137  0.426694


In [12]:
for (k1,k2), grupo in df.groupby(['key1','key']):
  print((k1,k2))
  print(grupo)

('a', 'one')
  key1  key     data1     data2
0    a  one -1.935819 -0.362509
4    a  one  2.679130  0.194852
('a', 'two')
  key1  key     data1     data2
1    a  two  0.286133 -0.252848
('b', 'one')
  key1  key     data1     data2
2    b  one  0.103624  0.410228
('b', 'two')
  key1  key     data1     data2
3    b  two  0.964137  0.426694


In [13]:
#gerando dicionario com uma parte dos dados

parte = dict(list(df.groupby('key1')))
parte['b']

,key1,key,data1,data2
2,b,one,0.103624,0.410228
3,b,two,0.964137,0.426694


In [14]:
df.dtypes

key1      object
key       object
data1    float64
data2    float64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, grupo in grouped:
  print(dtype)
  print(grupo)

float64
      data1     data2
0 -1.935819 -0.362509
1  0.286133 -0.252848
2  0.103624  0.410228
3  0.964137  0.426694
4  2.679130  0.194852
object
  key1  key
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


**Selecionando uma coluna ou um subconjunto de colunas**

In [16]:
df.groupby(['key1', 'key'])[['data2']].mean()

data2
key1 key          
a    one -0.083829
     two -0.252848
b    one  0.410228
     two  0.426694

In [17]:
s_grouped = df.groupby(['key1', 'key'])['data2']
s_grouped

In [18]:
s_grouped.mean()

key1  key
a     one   -0.083829
      two   -0.252848
b     one    0.410228
      two    0.426694
Name: data2, dtype: float64

**Agrupando com dicionário e séries**

In [19]:
people = pd.DataFrame(np.random.randn(5,5), columns=['a','b','c','d','e'], index=['Joe', 'Steve','Wes','Jim','Travis'])

In [20]:
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,0.492139,-0.469382,1.167805,0.632222,0.060261
Steve,0.064403,-0.374465,0.024672,1.454401,-0.865836
Wes,-0.061313,NaN,NaN,0.055208,-0.632025
Jim,-0.355018,-0.359726,0.109955,-0.141966,-0.056598
Travis,0.300781,1.290474,-0.379195,2.017070,-0.809854


In [21]:
#somar as colunas por grupo
mapping = {'a': 'red', 'b':'red', 'c':'blue', 'd':'blue','e':'red','f':'orange'}

In [22]:
by_columns = people.groupby(mapping, axis=1)
by_columns.sum()

,blue,red
Joe,1.800027,0.083018
Steve,1.479072,-1.175898
Wes,0.055208,-0.693338
Jim,-0.032011,-0.771341
Travis,1.637875,0.781402


In [23]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [24]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


**Agrupando com funções**

In [25]:
#agrupando por tamanho de string (contando string do índice)
people.groupby(len).sum()

,a,b,c,d,e
3,0.075808,-0.829109,1.277760,0.545464,-0.628361
5,0.064403,-0.374465,0.024672,1.454401,-0.865836
6,0.300781,1.290474,-0.379195,2.017070,-0.809854


In [26]:
key_list = ['one', 'one', 'one','two','two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.061313 -0.469382  1.167805  0.055208 -0.632025
  two -0.355018 -0.359726  0.109955 -0.141966 -0.056598
5 one  0.064403 -0.374465  0.024672  1.454401 -0.865836
6 two  0.300781  1.290474 -0.379195  2.017070 -0.809854

**Agrupando por níveis de índice**

In [27]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'], [1,3,5,1,3]], names=['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.811799 -2.055840  0.255858 -0.996674  1.265395
1      0.932959 -0.562367  1.576937 -1.283480  0.354622
2      0.665840  0.870122  0.505162 -1.273137  0.332166
3      0.023568  1.366967 -0.892020  0.393631  1.850397

In [28]:
hier_df.groupby(level = 'cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# Agregação de dados

In [29]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    2.200531
b    0.878085
Name: data1, dtype: float64

In [30]:
#Para usar uma função própria, utilize agg e passe a função entre parenteses
def peak_to_peak(arr):
  return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,4.614950,0.557361
b,0.860513,0.016465


In [31]:
grouped.describe()

data1                      ...     data2                    
     count      mean       std  ...       50%       75%       max
key1                            ...                              
a      3.0  0.343148  2.308003  ... -0.252848 -0.028998  0.194852
b      2.0  0.533880  0.608474  ...  0.418461  0.422577  0.426694

[2 rows x 16 columns]

**Aplicação de função nas colunas e aplicação de várias funções**

In [32]:
tips = pd.read_csv('/content/drive/MyDrive/dados_pandas/pydata-book-2nd-edition/examples/tips.csv')

In [33]:
#Acrescenta a porcentagem de gorjeta sobre o total da conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [34]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
#É possivel passar uma função descritiva como string
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [35]:
grouped_pct.agg(['mean','std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [36]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [37]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [38]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [39]:
#personalizar o nome das colunas
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [40]:
grouped.agg({'tip': np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [41]:
grouped.agg({'tip_pct':['min','max','mean','std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

**Devolvendo dados agregados sem índice de linha**

In [42]:
tips.groupby(['day','smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# Método apply: separar-aplicar-combinar genérico

Usando a base de dados de gorjetas vamos selecionar os cinco primeiros valores de `tip_pct`.

In [43]:
#Essa função seleciona as linhas com os maiores valores em uma coluna
def top (df, n=5, column = 'tip_pct'):
  return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [44]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [45]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [46]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [47]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [48]:
#mesma coisa da célula anterior
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

**Suprimindo as chaves de grupo**

In [49]:
#ocultar os índices hierarquicos
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


**Análise de quantis e de buckets**

In [50]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2':np.random.randn(1000)})

In [51]:
quartiles = pd.cut(frame.data1, 4)

In [52]:
quartiles[:10]

0    (-1.59, 0.0915]
1    (-3.278, -1.59]
2    (-1.59, 0.0915]
3    (-1.59, 0.0915]
4    (-1.59, 0.0915]
5    (-1.59, 0.0915]
6    (0.0915, 1.773]
7    (-1.59, 0.0915]
8    (0.0915, 1.773]
9    (0.0915, 1.773]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.278, -1.59] < (-1.59, 0.0915] < (0.0915, 1.773] <
                                    (1.773, 3.454]]

In [53]:
def get_stats(group):
  return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean':group.mean()}

In [54]:
grouped = frame.data2.groupby(quartiles)

In [55]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.278, -1.59]",-2.190906,2.762049,56.0,0.398914
"(-1.59, 0.0915]",-3.588539,3.755630,468.0,-0.007489
"(0.0915, 1.773]",-3.588061,2.585720,426.0,-0.049325
"(1.773, 3.454]",-2.189047,1.767329,50.0,-0.040461


In [56]:
#Obter só o número dos quantis

grouping = pd.qcut(frame.data1, 10, labels = False)

In [57]:
grouped = frame.data2.groupby(grouping)

In [58]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.469776,2.762049,100.0,0.183611
1,-2.838830,3.755630,100.0,-0.181261
2,-2.282797,2.392682,100.0,0.069689
3,-3.588539,2.667769,100.0,0.065604
4,-2.460184,3.266327,100.0,0.000170
5,-2.782258,2.467527,100.0,-0.108185
6,-2.415910,2.216701,100.0,0.057316
7,-2.577856,2.585720,100.0,-0.054502
8,-2.349218,2.148376,100.0,0.042811


**Preenchendo valores ausentes com valores específicos de grupo**

In [59]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.805462
2         NaN
3   -0.309089
4         NaN
5   -0.323185
dtype: float64

In [60]:
#preenchendo os valores ausentes com a media dos números
s.fillna(s.mean())

0    0.057729
1    0.805462
2    0.057729
3   -0.309089
4    0.057729
5   -0.323185
dtype: float64

In [61]:
#preenchendo de acordo com o grupo
states = ['Ohio', 'New Your', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [62]:
group_key = ['East'] * 4 + ['West'] * 4

In [63]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.359112
New Your     -0.323506
Vermont      -1.798261
Florida       1.150069
Oregon        0.376251
Nevada       -0.626136
California   -0.894977
Idaho        -0.618266
dtype: float64

In [64]:
#Definindo alguns valores como ausentes
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.359112
New Your     -0.323506
Vermont            NaN
Florida       1.150069
Oregon        0.376251
Nevada             NaN
California   -0.894977
Idaho              NaN
dtype: float64

In [65]:
#Preenchendo usando as medias dos grupos
fill_mean = lambda g: g.fillna(g.mean())

In [66]:
data.groupby(group_key).apply(fill_mean)

Ohio         -1.359112
New Your     -0.323506
Vermont      -0.177516
Florida       1.150069
Oregon        0.376251
Nevada       -0.259363
California   -0.894977
Idaho        -0.259363
dtype: float64

In [67]:
#Preencher com valores pré-definidos
fill_values = {'East': 0.5, 'West': -1}


In [68]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [69]:
data.groupby(group_key).apply(fill_func)

Ohio         -1.359112
New Your     -0.323506
Vermont       0.500000
Florida       1.150069
Oregon        0.376251
Nevada       -1.000000
California   -0.894977
Idaho        -1.000000
dtype: float64

**Amostragem aleatória e permutação**<br>

Uma maneira de construir um baralho em inglês

In [70]:
#Copas (Hearts), Espada (Spades), Paus(Clubs), Ouro (Diamonds)
suits = ['H', 'S', 'C', 'D']
card_val=(list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
  cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [71]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [72]:
#sortear uma mão de cinco baralhos

def draw(deck, n=5):
  return deck.sample(n)

draw(deck)

6H     6
7H     7
9D     9
QD    10
5C     5
dtype: int64

In [73]:
#Duas cartas aleatórias de cada naipe
get_suit = lambda card: card[-1] #a ultima letra é o naipe
deck.groupby(get_suit).apply(draw, n=2)

C  4C    4
   6C    6
D  AD    1
   9D    9
H  6H    6
   8H    8
S  4S    4
   8S    8
dtype: int64

In [74]:
#Outra forma de fazer a mesma coisa da célula anterior
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

AC      1
3C      3
AD      1
KD     10
AH      1
10H    10
2S      2
7S      7
dtype: int64

**Média ponderada de grupos e correlação**

In [78]:
df = pd.DataFrame({'category': ['a','a','a','a','b','b','b','b'], 'data': np.random.randn(8), 'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.329552,0.304628
1,a,0.517568,0.627854
2,a,-0.555931,0.779058
3,a,2.266695,0.336640
4,b,0.422754,0.187740
5,b,-0.600958,0.205142
6,b,0.028021,0.636073
7,b,-0.698813,0.747699


In [79]:
grouped = df.groupby('category')

In [82]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [83]:
grouped.apply(get_wavg)

category
a    0.368769
b   -0.308778
dtype: float64

In [84]:
#Preço das ações
close_px = pd.read_csv('/content/drive/MyDrive/dados_pandas/pydata-book-2nd-edition/examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [85]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [86]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [87]:
#Calcula a correlação
spx_corr = lambda x: x.corrwith(x['SPX'])

In [88]:
#Calcula as mudanças percentuais
rets = close_px.pct_change().dropna()

In [90]:
#agrupar as mudanças percentuais por ano
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [91]:
#Calculo de correlação entre as colunas
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

**Regessão linear nos grupos**

In [96]:
#usando a biblioteca statsmodel

def regress(data, yvar, xvars):
  Y = data[yvar]
  X = data[xvars]
  
  X['intercept'] = 1
  result = sm.OLS(Y, X).fit()
  return result.params

In [97]:
#regressão linear de AAPL

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# Tabelas pivôs e tabulação cruzada